## Triangular Arbitrage Scanner

### Libraries and Dependencies

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor

# Has private dependencies
from src.trading_logic import TradingLogicBase
from src.arbitrage.triangular import TriangularArbitrage
from src.market_actor import EchoMarketActor
from src.market_listener import MarketListenerBase, OrderBook
from src.market_listener.huobi import HuobiListener

### Market Listeners

In [2]:
SOL_USDC = HuobiListener(market=("SOL", "USDC"), headless_mode=True)
ETH_USDC = HuobiListener(market=("ETH", "USDC"), headless_mode=True)
SOL_ETH  = HuobiListener(market=("SOL", "ETH"),  headless_mode=True)

market_listeners = [SOL_USDC, ETH_USDC, SOL_ETH]

In [3]:
def subscribe(market_listeners: list[MarketListenerBase]):
    for market_listener in market_listeners:
        market_listener.subscribe()

        current_time = time.time()

        # timeout of 30s
        while (time.time() > current_time + 30) and not market_listener.rendered():
            time.sleep(.5) # Busy waiting

        assert market_listener.rendered()

In [4]:
subscribe(market_listeners)

In [5]:
mthread_executor = ThreadPoolExecutor(max_workers=3)

#### Testing Updating in Concurrency

In [10]:
for _ in range(10):
    current_prices = [
        f"{str(market_listener.get_current_ask()):<20}"
        for market_listener in market_listeners
    ]

    print(" , ".join(current_prices))
    time.sleep(1)

(14.7233, 100.06)    , (1208.33, 0.047)     , (0.011851, 2.4)     
(14.7214, 28.93)     , (1208.33, 0.047)     , (0.011852, 18.53)   
(14.7191, 104.02)    , (1208.33, 0.047)     , (0.011851, 0.69)    
(14.7147, 3.96)      , (1208.33, 0.047)     , (0.011851, 18.53)   
(14.7132, 2.4)       , (1208.33, 0.047)     , (0.011852, 18.53)   
(14.7115, 1.2)       , (1208.33, 0.047)     , (0.011852, 18.53)   
(14.7158, 32.88)     , (1208.32, 0.2836)    , (0.011852, 18.53)   
(14.7119, 100.06)    , (1208.33, 0.047)     , (0.011852, 18.53)   
(14.7109, 100.06)    , (1208.33, 0.047)     , (0.011851, 12.43)   
(14.7094, 28.92)     , (1208.33, 0.047)     , (0.011852, 18.53)   


### Market Actor
Actors abstract the order execution process

In [11]:
echo = EchoMarketActor(name="demo", transact_fee_rate=0.2 / 100)

### Trading Logic
Logic determines the price and size of the positions to enter

In [12]:
class CustomTradingLogic (TradingLogicBase):
    # A conservative approach should scan the 2nd entry (index=1) in the orderbook
    scan_entry_num = 0

    def get_size_from_portfolio(self, portfolio_value: float) -> float:
        return 1 # For demo purposes only

    def get_buy_price_from_orderbook(self, orderbook: OrderBook) -> float:
        ask_price, _, _ = orderbook.get_ask(self.scan_entry_num)
        return ask_price

    def get_buy_size_from_orderbook(self, orderbook: OrderBook) -> float:
        _, ask_size, _ = orderbook.get_ask(self.scan_entry_num)
        return ask_size

    def get_sell_price_from_orderbook(self, orderbook: OrderBook) -> float:
        bid_price, _, _ = orderbook.get_bid(self.scan_entry_num)
        return bid_price

    def get_sell_size_from_orderbook(self, orderbook: OrderBook) -> float:
        _, bid_size, _ = orderbook.get_bid(self.scan_entry_num)
        return bid_size

In [13]:
trading_logic = CustomTradingLogic()

### Arbitrage Machine

In [14]:
arb_scanner = TriangularArbitrage(
    origin_curr="USDC",
    trading_logic=trading_logic,
    market_listeners=market_listeners,
    market_actors=[echo, echo, echo]
)

In [15]:
# Sanity checks
print("Forward Path")
for node in arb_scanner.forward_path:
    print(node)

print("\nReverse Path")
for node in arb_scanner.reverse_path:
    print(node)

Forward Path
Long ('SOL', 'USDC')
Short ('SOL', 'ETH')
Short ('ETH', 'USDC')

Reverse Path
Long ('ETH', 'USDC')
Long ('SOL', 'ETH')
Short ('SOL', 'USDC')


### Arbitrage Scanning Latency Test

In [16]:
total_time, max_time, runs = 0, 0, 0

for _ in range(10000):
    try:
        current_time = time.time()

        # Updating orderbooks **********************************************************
        futures = [
            mthread_executor.submit(market_listener.update_orderbook)
            for market_listener in market_listeners
        ]
        
        for future in futures:
            future.result()

        # Scanning and executing trades
        origin_size = trading_logic.get_size_from_portfolio(1000) # Demo purposes only

        if arb_scanner.scan_and_execute(origin_size):
            break
        # ******************************************************************************

        elapsed = time.time() - current_time
        max_time = max(max_time, elapsed)
        total_time += elapsed
        runs += 1
    except Exception as exc:
        # Poor internet connection
        subscribe(market_listeners)
    
print(f"Average Latency: {total_time / runs:.2f}s")
print(f"Maximum Latency: {max_time:.2f}s")

Average Latency: 0.10s
Maximum Latency: 0.51s


### Shutting down Listeners

In [ ]:
for market_listener in market_listeners:
    market_listener.close()